# Linear Solver for Tableau Form

Packages required for the code:

In [1]:
import numpy as np

The function below is a simple implementation of a linear solver. It can solve LP and ILP problems using the following mechanisms:
* Primal Simplex Method
* Dual Simplex Method
* Gomory Cut

The function **linear_solver(tableau, beta, integer, verbose)** takes as input the following parameters:
* *tableau*: a numpy array of shape $n * m$ representing the entire tableau on which the operations will be carried out
* *beta*: an array representing the indexes of the basic variables *(Ex: beta=[3, 4, 5] representing $x_3, x_4, x_5$ as basic variables)*
* *integer*: boolean parameter in case it is needed an integer solution. Default is *False*
* *verbose*: boolean parameter use for printing infos and intermediate tableau. Default is *False*

In [13]:
def linear_solver(tableau, beta, integer=False, verbose=False):
    
    #print the starting tableau
    print('START TABLEAU:\n{}\n'.format(tableau))

    #number of rows in the tableau
    m = tableau.shape[0]
    #number of columns in the tableau
    n = tableau.shape[1]

    #flag for dual simplex method
    dual = False

    #final cases
    unbounded = False
    infeasible = False
    optimal = False

    while optimal == False and unbounded == False and infeasible == False:
        #get the vector of costs
        costs = [tableau[0][c] for c in range(1, n)]
        #print('Costs vector: ', costs)
        #verify if all the costs are >= 0, thus the tableau is in optimal form
        if all(c >= 0 for c in costs):
            #pick the b vector
            b = [tableau[i][0] for i in range(1, m)]            
            #verifiy if it is necessary to apply gomory's cut
            if integer: 
                if all(elem == int(elem) for elem in b):
                    optimal = True
                    if verbose:
                        print('\n***********Obtain optimal tableau**************\n')
                    break
                else:
                    #apply gomory cut on the first fractional variable (fract is the index)
                    fract = 0
                    for i in range(1, m):
                        if tableau[i][0] != int(tableau[i][0]):
                            fract = i
                            break

                    #print the tableau with the gomory cut
                    if verbose:
                        print('TABLEAU WITH GOMORY CUT ON ROW=', fract)

                    #add the new row
                    tableau = np.vstack((tableau, np.array([0.]*n)))
                    #add the new column
                    tableau = np.hstack((tableau, np.zeros((m+1, 1), dtype=float)))
                    tableau[m][n] = 1.
                    
                    #update the order of the tableau and the vector beta
                    m = tableau.shape[0]
                    n = tableau.shape[1]
                    beta.append(n-1)

                    #update the value in the new row added
                    for i in range(n-1):
                        if tableau[fract][i] >= 0:
                            tableau[m-1][i] = -(tableau[fract][i] - int(tableau[fract][i]))
                        else:
                            tableau[m-1][i] = -(tableau[fract][i] - int(tableau[fract][i]-1))
                    if verbose:
                        print(tableau)

                    #update the cost vector
                    costs = [tableau[0][c] for c in range(1, n)]

                    #update flag in order to apply dual simplex method
                    dual = True
            #if there is no need for the solution to be integer then it is optimal
            else:
                optimal = True
                break

        #if the gomory cuts have been applied, continue with the dual simplex problem
        if dual:
            #index of basic variable that will leave the basis
            t = 1
            #find the first negative variable with b < 0
            for i in range(1, m):
                if i != 0 and tableau[i][0] < 0:
                    t = i
                    break
            if all(tableau[t][i] >= 0 for i in range(1, n)):
                infeasible = True
                break
            else:
                #choose the variable that will enter the basis
                min = 100000
                h = 1
                for i in range(1, n):
                    if tableau[t][i] < 0 and tableau[0][i] / abs(tableau[t][i]) < min:
                        min = tableau[0][i] / abs(tableau[t][i])
                        h = i
                if verbose:
                    print('Pivot operation on the cell: {}\n'.format((t, h)))
                
                #PIVOT OPERATION
                pivot = tableau[t][h]
                #divide the pivot row by the pivot element
                for i in range(n):
                    tableau[t][i] /= pivot
                #update all the other rows
                save = pivot
                for i in range(m):
                    if i != t and tableau[i][h] != 0:
                        save = tableau[i][h]
                        for j in range(n):
                            tableau[i][j] -= save*tableau[t][j]
                #END PIVOT OPERATION
                #update the vector beta, containing the indices of the basis variables
                beta[t-1] = h
        else:
            #index of the non basic variable
            h = 0
            #Find the first cost < 0 and choose a non basic variable
            for i, c in enumerate(tableau[0]):
                if i != 0 and c < 0:
                    h = i
                    #print('Variable of index {} will enter the basis'.format(h))
                    break
            #check if all the a[i, h] are < 0 so the problem is unbounded
            if all(tableau[i, h] < 0 for i in range(m)):
                unbounded = True
                break
            else:
                #choose the variable that will leave the basis
                min = 100000
                t = 1
                for i in range(1, m):
                    if tableau[i][h] > 0 and tableau[i][0] / tableau[i][h] < min:
                        min = tableau[i][0] / tableau[i][h]
                        t = i
                if verbose:
                    print('Pivot operation on the cell: {}\n'.format((t, h)))

                #PIVOT OPERATION
                pivot = tableau[t][h]
                #divide the pivot row by the pivot element
                for i in range(n):
                    tableau[t][i] /= pivot
                #update all the other rows
                save = pivot
                for i in range(m):
                    if i != t and tableau[i][h] != 0:
                        save = tableau[i][h]
                        for j in range(n):
                            tableau[i][j] -= save*tableau[t][j]
                #END PIVOT OPERATION
                #update the vector beta, containing the indices of the basis variables
                beta[t-1] = h

        #Print the intermediate tableau
        if verbose:
            print('TABLEAU:\n{}\n'.format(tableau))

    #solution
    x = [0]*(n - 1)
    for i, b in enumerate(beta):
        x[b-1] = tableau[i+1][0]              
            

    if optimal:
        print('The tableau has an optimal solution x = ', x)
    elif unbounded:
        print('The tableau is unbounded')
    elif infeasible:
        print('The problem is infeasible')

### Test 1: Primal Simplex Method

Creation of a simple tableau and execution of the code.

In [10]:
tableau = np.array([[0., -1., -1., 0., 0.], [24., 6., 4., 1., 0.], [6., 3., -2., 0., 1.]])
beta = np.array([1, 2])

#primal simple method only
linear_solver(tableau, beta)

START TABLEAU:
[[ 0. -1. -1.  0.  0.]
 [24.  6.  4.  1.  0.]
 [ 6.  3. -2.  0.  1.]]

The tableau has an optimal solution x =  [0, 6.0, 0, 18.0]


### Test 2: ILP Problem
In this case, first the Primal Simplex Method will be use, then the combination of Gomory Cut and Dual Simplex Method will find, if exists, the optimal integer solution

In [11]:
tableau2 = np.array([[0, -4, -5, 0, 0, 0], [8, 2, 2, 1, 0, 0], [7, 1, 3, 0, 1, 0], [5, 2, 1, 0, 0, 1]], dtype='float')
beta2 = [3, 4, 5]
linear_solver(tableau2, beta2, integer=True)

START TABLEAU:
[[ 0. -4. -5.  0.  0.  0.]
 [ 8.  2.  2.  1.  0.  0.]
 [ 7.  1.  3.  0.  1.  0.]
 [ 5.  2.  1.  0.  0.  1.]]

The tableau has an optimal solution x =  [1.0, 2.0, 2.0, 0.0, 1.0, 0, 0]


### Test 3

In [ ]:
#TODO: there is a problem in this example, about fractional value that doesn't stop the algorithm when the optimal tableau is reached.
tableau3 = np.array([[0, -3, -2, 0, 0, 0], [7, 2, 1, 1, 0, 0], [8, 3, 2, 0, 1, 0], [6, 1, 1, 0, 0, 1]], dtype='float')
beta3 = [3, 4, 5]
linear_solver(tableau3, beta3, integer=True)

### Test 4

In [12]:
tableau4 = np.array([[0, -2, -1, 0, 0], [0, 3, -2, 1, 0], [6, 1, 2, 0, 1]], dtype='float')
beta4 = [3, 4]
linear_solver(tableau4, beta4, integer=True, verbose=True)

START TABLEAU:
[[ 0. -2. -1.  0.  0.]
 [ 0.  3. -2.  1.  0.]
 [ 6.  1.  2.  0.  1.]]

Pivot operation on the cell: (1, 1)

TABLEAU:
[[ 0.          0.         -2.33333333  0.66666667  0.        ]
 [ 0.          1.         -0.66666667  0.33333333  0.        ]
 [ 6.          0.          2.66666667 -0.33333333  1.        ]]

Pivot operation on the cell: (2, 2)

TABLEAU:
[[ 5.25   0.     0.     0.375  0.875]
 [ 1.5    1.     0.     0.25   0.25 ]
 [ 2.25   0.     1.    -0.125  0.375]]

TABLEAU WITH GOMORY CUT ON ROW= 1
[[ 5.25   0.     0.     0.375  0.875  0.   ]
 [ 1.5    1.     0.     0.25   0.25   0.   ]
 [ 2.25   0.     1.    -0.125  0.375  0.   ]
 [-0.5   -0.    -0.    -0.25  -0.25   1.   ]]
Pivot operation on the cell: (3, 3)

TABLEAU:
[[ 4.5  0.   0.   0.   0.5  1.5]
 [ 1.   1.   0.   0.   0.   1. ]
 [ 2.5  0.   1.   0.   0.5 -0.5]
 [ 2.   0.   0.   1.   1.  -4. ]]

TABLEAU WITH GOMORY CUT ON ROW= 2
[[ 4.5  0.   0.   0.   0.5  1.5  0. ]
 [ 1.   1.   0.   0.   0.   1.   0. ]
 [ 2.5  0.